<h1> Predicting Drought Using Meteorological and Soil data </h1>

<h2>Software engineering seminar - 10400</h2>
<h2>Last edit - May 10th, 2021</h2>

<h3>Background</h3>
Lots of text in here..

<h3>Hypothesis</h3>
Lots of text here as well...

In [ ]:
import numpy as numpy
import pandas as pandas
import seaborn as seaborn
import matplotlib.pylab as plot_library
from sklearn import metrics, neighbors
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, plot_confusion_matrix, roc_auc_score, roc_curve
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import ComplementNB
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier


<h3>Dataset Description</h3>
Dataset taken from Kaggle. <br>
Derived from.. etc..

In [ ]:
#Import meteorological data.
train_frame = pandas.read_csv('resources/train_timeseries.csv')
validation_frame = pandas.read_csv('resources/validation_timeseries.csv')
test_frame = pandas.read_csv('resources/test_timeseries.csv')
#Import soil data.
soil_frame = pandas.read_csv('resources/soil_data.csv')

In [ ]:
#Print shapes of data.
print('The shapes of our dataframes are')
print('train_frame', train_frame.shape)
print('validation_frame', validation_frame.shape)
print('test_frame', test_frame.shape)
print('soil_frame', soil_frame.shape)

<h3>Data Manipulating</h3>
<h4> Empty values </h4>
Check every imported frame for empty values. Because the target value is something measured and there <br>
is plenty of data, empty fields will be removed rather than filled.

In [ ]:
#train_frame manipulations.
print("train_frame\n")
print("count - \n", train_frame.count())
print("max - \n", train_frame.max())
print("min - \n", train_frame.min())
print("empty amount - \n", train_frame.isna().any())
print("empty total - \n", train_frame.isna().sum())
print("empty precent - \n", train_frame.isna().sum() / len(train_frame.index) * 100)

In [ ]:
#train_frame remove empty values.
train_frame = train_frame.dropna(subset=["score"])


In [ ]:
#validation_frame manipulations.
print("validation_frame\n")
print("count - \n", validation_frame.count())
print("max - \n", validation_frame.max())
print("min - \n", validation_frame.min())
print("empty amount - \n", validation_frame.isna().any())
print("empty total - \n", validation_frame.isna().sum())
print("empty precent - \n", validation_frame.isna().sum() / len(validation_frame.index) * 100)

In [ ]:
#validation_frame remove empty values.
validation_frame = validation_frame.dropna(subset=["score"])


In [ ]:
#test_frame manipulations.
print("test_frame\n")
print("count - \n", test_frame.count())
print("max - \n", test_frame.max())
print("min - \n", test_frame.min())
print("empty amount - \n", test_frame.isna().any())
print("empty total - \n", test_frame.isna().sum())
print("empty precent - \n", test_frame.isna().sum() / len(test_frame.index) * 100)

In [ ]:
#test_frame remove empty values.
test_frame = test_frame.dropna(subset=["score"])


In [ ]:
#soil_frame manipulations.
print("soil_frame\n")
print("count - \n", soil_frame.count())
print("max - \n", soil_frame.max())
print("min - \n", soil_frame.min())
print("empty amount - \n", soil_frame.isna().any())
print("empty total - \n", soil_frame.isna().sum())
print("empty precent - \n", soil_frame.isna().sum() / len(soil_frame.index) * 100)

In [ ]:
#soil_frame doesn't have empty cells.
#Print update shapes of data.
print('The updated shapes of our dataframes are')
print('train_frame', train_frame.shape)
print('validation_frame', validation_frame.shape)
print('test_frame', test_frame.shape)
print('soil_frame', soil_frame.shape)



<h4> Merging Data </h4>
Merging the meteorological data with the soil data.

In [ ]:
#Merge train_frame and soil_frame.
train_soil_frame = pandas.merge(train_frame, soil_frame, how='inner', on='fips')
train_soil_frame.head()

In [ ]:
#Merge validation_frame and soil_frame.
validation_soil_frame = pandas.merge(validation_frame, soil_frame, how='inner', on='fips')
validation_soil_frame.head()

In [ ]:
#Merge test_frame and soil_frame.
test_soil_frame = pandas.merge(test_frame, soil_frame, how='inner', on='fips')
test_soil_frame.head()

In [ ]:
#Merge all frames together.
total_frame = pandas.concat([train_soil_frame, validation_soil_frame, test_soil_frame])
total_frame.shape

Remove 'fips' and 'date' columns as we do not need their influence in the calculations. <br>
We want to find the correlation between meteorological and soil data to the drought 'score' column.

In [ ]:
#Remove fips column
del total_frame["fips"]

#Remove date column
del total_frame["date"]

#Remove latitude column
del total_frame["lat"]

#Remove longitude column
del total_frame["lon"]

<h4>Finished</h4>
Now we have the finished data frame to work on.

In [ ]:
total_frame

<h4>Correlation Testing</h4>
Test whether there are any columns which poorly affect the score target value.

In [ ]:
#Pearson test for total correlation checking
fig, ax = plot_library.subplots(figsize=(75, 50))
seaborn.heatmap(total_frame.corr(method='pearson'), annot=True, ax=ax, cmap=plot_library.cm.Blues)
plot_library.savefig('total_frame_corr.png')

Splitting the columns to find the biggest correlation with score.

In [ ]:
#Meteorological and drought
fig, ax = plot_library.subplots(figsize=(20, 15))
seaborn.heatmap(total_frame[['PRECTOT', 'PS',
                                        'QV2M', 'T2M',
                                        'T2MDEW', 'T2MWET',
                                        'T2M_MAX', 'T2M_MIN',
                                        'T2M_RANGE', 'TS',
                                        'WS10M', 'WS10M_MAX',
                                        'WS10M_MIN', 'WS10M_RANGE',
                                        'WS50M', 'WS50M_MAX',
                                        'WS50M_MIN', 'WS50M_RANGE',
                                        'score']].corr(method='pearson'),
                           annot=True, ax=ax, cmap=plot_library.cm.Blues)
plot_library.savefig('meteorolgical_corr.png')

In [ ]:
#Soil and drought
fig, ax = plot_library.subplots(figsize=(25, 20))
seaborn.heatmap(total_frame[['elevation', 'slope1',
                                        'slope2', 'slope3',
                                        'slope4', 'slope5',
                                        'slope6', 'slope7',
                                        'slope8', 'aspectN',
                                        'aspectW', 'aspectE',
                                        'aspectS', 'aspectUnknown',
                                        'WAT_LAND', 'NVG_LAND',
                                        'URB_LAND', 'GRS_LAND',
                                        'FOR_LAND', 'CULTRF_LAND',
                                        'CULTIR_LAND', 'CULT_LAND',
                                        'SQ1', 'SQ2',
                                        'SQ3', 'SQ4',
                                        'SQ5', 'SQ6',
                                        'SQ7', 'score']].corr(method='pearson'),
                           annot=True, ax=ax, cmap=plot_library.cm.Blues)
plot_library.savefig('soil_corr.png')

In [ ]:
#Higher correlation numbers closeup.
fig, ax = plot_library.subplots(figsize=(6, 5))
seaborn.heatmap(total_frame[['T2M_RANGE', 'T2M_MAX','elevation', 'NVG_LAND',
                                        'GRS_LAND', 'CULTRF_LAND', 'score']].corr(method='pearson'),
                           annot=True, ax=ax, cmap=plot_library.cm.Blues)
plot_library.savefig('closeup_corr.png')


<h4> Imbalanced Dataset <h4>

In [ ]:
score_balance = []

for i in range(0,6):
    tempRows = total_frame[(total_frame['score'] >= i) & (total_frame['score'] < (i + 1))]
    score_balance.append(len(tempRows))

print("Distribution of score classes. From 0 to 5.")
for j in score_balance:
    print()
    print(f"Amount is {j}. Which are {j / sum(score_balance) * 100:.3}%.")



<h4>Separation of target and features<h4>

In [ ]:
#Classification preparation
# Extract the training and test data
data = total_frame
X = data[['PRECTOT', 'PS', 'QV2M', 'T2M', 'T2MDEW', 'T2MWET',
          'T2M_MAX', 'T2M_MIN', 'T2M_RANGE', 'TS', 'WS10M', 'WS10M_MAX',
          'WS10M_MIN', 'WS10M_RANGE', 'WS50M', 'WS50M_MAX', 'WS50M_MIN',
          'WS50M_RANGE', 'elevation', 'slope1', 'slope2', 'slope3', 'slope4',
          'slope5', 'slope6', 'slope7', 'slope8', 'aspectN', 'aspectW',
          'aspectE', 'aspectS', 'aspectUnknown', 'WAT_LAND', 'NVG_LAND',
          'URB_LAND', 'GRS_LAND', 'FOR_LAND', 'CULTRF_LAND', 'CULTIR_LAND',
          'CULT_LAND', 'SQ1', 'SQ2', 'SQ3', 'SQ4', 'SQ5', 'SQ6', 'SQ7']]
y = data['score'].apply(numpy.floor)

<h4>Separation into training and test sets<h4>

In [ ]:
#Split into training and test. 30/70.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30)

<h3>Algorithms</h3>
Both regression and classifier algorithms will be utilized.

<h4>Random Forest</h4>


Regression

In [ ]:
#Regression preparation
#Extract the training and test data
data_reg = total_frame.values
X_reg = data_reg[:, 1:]  # all rows, no label
y_reg = data_reg[:, 18]  # all rows, label only

In [ ]:
#Split into training and test. 30/70.
X_reg_train, X_reg_test, y_reg_train, y_reg_test = train_test_split(X_reg, y_reg, test_size=0.30)

In [ ]:
# Establish model with different amounts of estimators.
RFRmodel = RandomForestRegressor(n_jobs=-1)
#5 estimators
RFRmodel.set_params(n_estimators=5)
RFRmodel.fit(X_reg_train, y_reg_train)

In [ ]:
#Score value for determining accuracy.
RFR_score = RFRmodel.score(X_reg_test, y_reg_test)
print(f'Score value for 5 estimators is {RFR_score}')

In [ ]:
#Root median square as a measure of accuracy.
RFR_y_pred = RFRmodel.predict(X_reg_test)
print("RMS: %r " % numpy.sqrt(numpy.mean((y_train - y_test) ** 2)))

Classification

In [ ]:
#Establish classification model.
RFC_model = RandomForestClassifier(n_jobs=-1)


In [ ]:
RFC_model.set_params()
RFC_model.fit(X_train, y_train)
RFC_y_pred = RFC_model.predict(X_test)


In [ ]:
print("Training accuracy", RFC_model.score(X_train, y_train))
print("Testing accuracy", RFC_model.score(X_test, y_test))
RFC_score = metrics.accuracy_score(y_test, RFC_y_pred)
print("Accuracy:", RFC_score)

In [ ]:
print(classification_report(y_test, RFC_y_pred))

In [ ]:
print(confusion_matrix(y_test, RFC_y_pred))

In [ ]:
display = plot_confusion_matrix(RFC_model, X_test, y_test,
                                cmap=plot_library.cm.Blues ,
                                normalize=None)

plot_library.savefig('confusionRFC.png')

<h3> Multinomial Naive Bayes <h3>


In [ ]:
# Scale the data between 0 and 1.
scaler = MinMaxScaler(feature_range=[0,1])
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
CNB_model = ComplementNB()
CNB_model.fit(X_train, y_train)
CNB_y_pred = CNB_model.predict(X_test)

In [ ]:
print("Training accuracy", CNB_model.score(X_train, y_train))
print("Testing accuracy", CNB_model.score(X_test, y_test))
CNB_score = metrics.accuracy_score(y_test, CNB_y_pred)
print("Accuracy:", CNB_score)

In [ ]:
print(classification_report(y_test, CNB_y_pred))

In [ ]:
print(confusion_matrix(y_test, CNB_y_pred))

In [ ]:
display = plot_confusion_matrix(CNB_model, X_test, y_test,
                                cmap=plot_library.cm.Blues ,
                                normalize=None)

plot_library.savefig('confusionNB.png')

<h3>Logistic Regression<h3>

In [ ]:
LR_model = LogisticRegression(solver='sag', multi_class='multinomial')
LR_model.fit(X_train, y_train)
LR_y_pred = LR_model.predict(X_test)

In [ ]:
print("Training accuracy", LR_model.score(X_train, y_train))
print("Testing accuracy", LR_model.score(X_test, y_test))
LR_score = metrics.accuracy_score(y_test, LR_y_pred)
print("Accuracy:", LR_score)

In [ ]:
print(classification_report(y_test, LR_y_pred))

In [ ]:
print(confusion_matrix(y_test, LR_y_pred))

In [ ]:
display = plot_confusion_matrix(LR_model, X_test, y_test,
                                cmap=plot_library.cm.Blues ,
                                normalize=None)

plot_library.savefig('confusionLR.png')

In [ ]:
# K nearest neighbors redacted for taking way too long to predict values.

In [ ]:
#scaling to decrease running time.
# scaler = StandardScaler()
# scaler.fit(X_train)
# X_train_tran = scaler.transform(X_train)
# X_test_tran = scaler.transform(X_test)

In [ ]:
# KNN_model = neighbors.KNeighborsClassifier(n_jobs=-1, n_neighbors=1)
# KNN_model.fit(X_train_tran, y_train)
# KNN_y_pred = KNN_model.predict(X_test_tran)

In [ ]:
# print("Training accuracy", KNN_model.score(X_train_tran, y_train))
# print("Testing accuracy", KNN_model.score(X_test_tran, y_test))
# print("Accuracy:", metrics.accuracy_score(y_test, KNN_y_pred))

In [ ]:
# print(classification_report(y_test, KNN_y_pred))

In [ ]:
# print(confusion_matrix(y_test, KNN_y_pred))

In [ ]:
# display = plot_confusion_matrix(KNN_model, X_test_tran, y_test,
#                                 cmap=plot_library.cm.Blues ,
#                                 normalize=None)
#
# plot_library.savefig('confusionKNN.png')

<h3>Retry with different params<h3>

In [85]:
#Classification preparation
# Extract the training and test data
data = total_frame
X = data[['T2M_RANGE', 'T2M_MAX','elevation',
          'NVG_LAND', 'GRS_LAND', 'CULTRF_LAND']]
y = data['score'].apply(numpy.floor)

<h4>Separation into training and test sets<h4>

In [86]:
#Split into training and test. 30/70.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30)

<h3>Algorithms</h3>
Both regression and classifier algorithms will be utilized.

<h4>Random Forest</h4>


Regression

In [82]:
#Regression preparation
#Extract the training and test data
data_reg = total_frame.values
X_reg = data_reg[:, 1:]  # all rows, no label
y_reg = data_reg[:, 7]  # all rows, label only

In [83]:
#Split into training and test. 30/70.
X_reg_train, X_reg_test, y_reg_train, y_reg_test = train_test_split(X_reg, y_reg, test_size=0.30)

In [84]:
# Establish model with different amounts of estimators.
RFRmodel = RandomForestRegressor(n_jobs=-1)
#5 estimators
RFRmodel.set_params(n_estimators=5)
RFRmodel.fit(X_reg_train, y_reg_train)

KeyboardInterrupt: 

In [ ]:
#Score value for determining accuracy.
RFR_score = RFRmodel.score(X_reg_test, y_reg_test)
print(f'Score value for 5 estimators is {RFR_score}')

In [ ]:
#Root median square as a measure of accuracy.
RFR_y_pred = RFRmodel.predict(X_reg_test)
print("RMS: %r " % numpy.sqrt(numpy.mean((y_train - y_test) ** 2)))

Classification

In [87]:
#Establish classification model.
RFC_model = RandomForestClassifier(n_jobs=-1)


In [88]:
RFC_model.set_params()
RFC_model.fit(X_train, y_train)
RFC_y_pred = RFC_model.predict(X_test)


In [89]:
print("Training accuracy", RFC_model.score(X_train, y_train))
print("Testing accuracy", RFC_model.score(X_test, y_test))
RFC_score = metrics.accuracy_score(y_test, RFC_y_pred)
print("Accuracy:", RFC_score)

Training accuracy 0.9998041482819778
Testing accuracy 0.6439699739018369
Accuracy: 0.6439699739018369


In [90]:
print(classification_report(y_test, RFC_y_pred))

              precision    recall  f1-score   support

         0.0       0.71      0.91      0.80    691175
         1.0       0.23      0.10      0.14    152193
         2.0       0.19      0.08      0.11     90883
         3.0       0.17      0.07      0.09     55138
         4.0       0.13      0.04      0.07     25794
         5.0       0.15      0.05      0.07      6728

    accuracy                           0.64   1021911
   macro avg       0.26      0.21      0.21   1021911
weighted avg       0.54      0.64      0.58   1021911



In [91]:
print(confusion_matrix(y_test, RFC_y_pred))


[[631480  30852  15753   8870   3491    729]
 [125068  14737   6831   3722   1464    371]
 [ 70964   8379   6837   3135   1280    288]
 [ 40786   5568   3832   3595   1113    244]
 [ 18350   2816   1888   1447   1122    171]
 [  4584    763    525    336    211    309]]


<h3> Multinomial Naive Bayes <h3>


In [92]:
# Scale the data between 0 and 1.
scaler = MinMaxScaler(feature_range=[0,1])
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [93]:
CNB_model = ComplementNB()
CNB_model.fit(X_train, y_train)
CNB_y_pred = CNB_model.predict(X_test)

In [94]:
print("Training accuracy", CNB_model.score(X_train, y_train))
print("Testing accuracy", CNB_model.score(X_test, y_test))
CNB_score = metrics.accuracy_score(y_test, CNB_y_pred)
print("Accuracy:", CNB_score)

Training accuracy 0.45419858693195136
Testing accuracy 0.4541373955266163
Accuracy: 0.4541373955266163


In [95]:
print(classification_report(y_test, CNB_y_pred))

              precision    recall  f1-score   support

         0.0       0.72      0.62      0.67    691175
         1.0       0.13      0.00      0.00    152193
         2.0       0.15      0.13      0.14     90883
         3.0       0.07      0.41      0.12     55138
         4.0       0.03      0.02      0.03     25794
         5.0       0.01      0.00      0.00      6728

    accuracy                           0.45   1021911
   macro avg       0.19      0.20      0.16   1021911
weighted avg       0.53      0.45      0.47   1021911



In [96]:
print(confusion_matrix(y_test, CNB_y_pred))

[[429072    282  37001 208992  15620    208]
 [ 83696     50  14135  50979   3248     85]
 [ 44407     18  11506  33073   1812     67]
 [ 23016     19   8023  22821   1213     46]
 [ 10114      6   3769  11249    633     23]
 [  2676      0    642   3241    163      6]]


<h3>Logistic Regression<h3>

In [97]:
LR_model = LogisticRegression(solver='sag', multi_class='multinomial')
LR_model.fit(X_train, y_train)
LR_y_pred = LR_model.predict(X_test)

In [98]:
print("Training accuracy", LR_model.score(X_train, y_train))
print("Testing accuracy", LR_model.score(X_test, y_test))
LR_score = metrics.accuracy_score(y_test, LR_y_pred)
print("Accuracy:", LR_score)

Training accuracy 0.6759278946946831
Testing accuracy 0.6763084065050675
Accuracy: 0.6763084065050675


In [99]:
print(classification_report(y_test, LR_y_pred))

c:\users\nahum\documents\projects\python\seminar\venv\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\nahum\documents\projects\python\seminar\venv\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

         0.0       0.68      1.00      0.81    691175
         1.0       0.00      0.00      0.00    152193
         2.0       0.22      0.00      0.00     90883
         3.0       0.17      0.00      0.00     55138
         4.0       0.00      0.00      0.00     25794
         5.0       0.00      0.00      0.00      6728

    accuracy                           0.68   1021911
   macro avg       0.18      0.17      0.13   1021911
weighted avg       0.49      0.68      0.55   1021911



c:\users\nahum\documents\projects\python\seminar\venv\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [100]:
print(confusion_matrix(y_test, LR_y_pred))

[[691078      0     44     53      0      0]
 [152147      0     19     27      0      0]
 [ 90831      0     24     28      0      0]
 [ 55098      0     15     25      0      0]
 [ 25775      0      7     12      0      0]
 [  6727      0      0      1      0      0]]
